In [8]:
import xarray as xr 
import numpy as np
import os 
from glob import glob

In [2]:
# TODO: Think about using patching solely due to difference in shapes 
# of the 3-km and 1-km grids.

def load_data(date, init_time, time_ind, ens_ind=1):
    #/work/brian.matilla/WOFS_2021/summary_files/WOFS_JET/WOFS_1km_downscaled/
    base_path = '/work/brian.matilla/WOFS_2021/summary_files/WOFS_JET/WOFS_1KM/'
    file_path = glob(os.path.join(base_path, date, init_time, f'wofs_ENS_{time_ind}*'))[0]
    ds_1km = xr.load_dataset(file_path, decode_times=False)
    dbz_1km = ds_1km['comp_dz'].values[ens_ind,:,:]
    
    base_path = '/work/mflora/SummaryFiles/'
    file_path = glob(os.path.join(base_path, date, init_time, f'wofs_ENS_{time_ind}*'))[0]
    ds_3km = xr.load_dataset(file_path, decode_times=False)
    dbz_3km = ds_3km['comp_dz'].values[ens_ind,:,:]
    
    return dbz_1km, dbz_3km, ds_1km, ds_3km

In [3]:
dates = os.listdir('/work/brian.matilla/WOFS_2021/summary_files/WOFS_JET/WOFS_1KM/')
dates.remove('sens_test_dates')

In [4]:
date = '20210504'
init_time = '2300'
time_ind = '03'
ens_ind = 4

dbz_1km, dbz_3km, ds_1km, ds_3km = load_data(date, init_time, time_ind, ens_ind)

In [9]:
# Resample 1km to the 3km grid before patching. 
# Make sure the 1-km and 3-km span the same lat/lon space. 
xlat = ds_1km['xlat'].values
xlat_min, xlat_max = np.min(xlat), np.max(xlat)

xlon = ds_1km['xlon'].values
xlon_min, xlon_max = np.min(xlon), np.max(xlon)

xlat_3km = ds_3km['xlat'].values
xlon_3km = ds_3km['xlon'].values

# lat = y, lon = x 
jmax = np.argmin(np.abs(xlat_3km[:,0] - xlat_max))
jmin = np.argmin(np.abs(xlat_3km[:,0] - xlat_min))

imax = np.argmin(np.abs(xlon_3km[0,:] - xlon_max))
imin = np.argmin(np.abs(xlon_3km[0,:] - xlon_min))

dbz_3km_res = dbz_3km[jmin:jmax+1, imin:imax+1]

In [10]:
dbz_3km_res.shape

(136, 144)